In [28]:
import os
import pandas as pd
import sys
import torch
import numpy as np
sys.path.append("../")
# sys.path.append("../hms_pipeline")

from hms_pipeline.spec_dataset import SpecDataset, infer_spec_dataset
from hms_pipeline.eeg_model import SpecModel, SpecVitModel
from hms_pipeline.kaggle_kl_div import eval_subm, eval_subm_g10, score

from hms_pipeline.eeg_model import EegInferModule


In [29]:
def team_eval(df_subm, df_labels_all_path = "/home/maxc/workspace/kaggle-hms/notebooks/train_fold_irr_mark.csv"):
    df_labels_all_path = "/home/maxc/workspace/kaggle-hms/notebooks/train_fold_irr_mark.csv"
    df_labels_all = pd.read_csv(df_labels_all_path)
    df_subm = df_subm.rename({'seizure_vote':'seizure_pred','lpd_vote':'lpd_pred','gpd_vote':'gpd_pred','lrda_vote':'lrda_pred',
                                    'grda_vote':'grda_pred','other_vote':'other_pred',
                                    }, axis=1)
    df_subm = df_subm.merge(df_labels_all[['eeg_id','eeg_sub_id','label_id','patient_id',
                                                'seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote',
                                                'fold','n_votes']], on=['eeg_id','eeg_sub_id'], how='left')
    
    def get_team_score(df_subm, calc_mode = 'high_vote'):
        if calc_mode  == 'high_vote':
            df_subm   = df_subm[df_subm['n_votes']>9].reset_index(drop=True)
        # df_oof_chen   = df_subm.groupby('eeg_id').first().reset_index(drop=True)
        df_oof_chen =  df_oof_chen.groupby('eeg_id').apply(lambda x: x.iloc[len(x) // 2]).reset_index(drop=True) 
        col_labels = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
        df_labels           = df_oof_chen.copy()
        df_labels           = df_labels[col_labels]
        df_labels['id']     = np.arange(len(df_labels))


        df_oof_chen         = df_oof_chen[['seizure_pred','lpd_pred','gpd_pred','lrda_pred','grda_pred','other_pred',]]
        df_oof_chen         = df_oof_chen.rename({'seizure_pred':'seizure_vote','lpd_pred':'lpd_vote','gpd_pred':'gpd_vote','lrda_pred':'lrda_vote',
                                                'grda_pred':'grda_vote','other_pred':'other_vote',
                                                    }, axis=1)
        df_oof_chen['id']   = np.arange(len(df_oof_chen))


        print(df_labels.shape, df_oof_chen.shape)
        metrics_flg1_vote   = score(solution=df_labels.copy(), submission=df_oof_chen.copy(), 
                                                                        row_id_column_name='id')
        return metrics_flg1_vote
    
    all_score = get_team_score(df_subm, calc_mode = 'all')
    high_score = get_team_score(df_subm, calc_mode = 'high_vote')
    print(all_score, high_score)

In [30]:
df_subm_path = "/home/maxc/workspace/kaggle-hms/notebooks/maxc_ensemble_preds.csv"
df_subm = pd.read_csv(df_subm_path)
df_subm.head()

,eeg_id,eeg_sub_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1001487592,0,0.016803,0.223291,0.717147,0.003824,0.008070,0.030865
1,1001487592,1,0.016345,0.199719,0.740909,0.003590,0.007927,0.031509
2,1001487592,2,0.016444,0.222757,0.720017,0.003235,0.007517,0.030030
3,1001487592,3,0.018032,0.255565,0.663758,0.005393,0.014831,0.042421
4,1003458521,0,0.542397,0.157613,0.002799,0.112832,0.005545,0.178814


In [31]:
my_subm_df = pd.read_csv("/home/maxc/workspace/kaggle-hms/notebooks/maxc_ensemble_preds.csv")
# my_subm_df = my_subm_df.head(200)
team_eval(my_subm_df)

(17089, 7) (17089, 7)
(5939, 7) (5939, 7)
0.47374749401728633 0.23392707130197804
